In [1]:
import pandas as pd 
import seaborn as sns
#import missingno as mg 
from matplotlib import pyplot as plt
from sklearn import preprocessing 
import re
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
import researchpy as rp
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.svm import SVC
from sklearn import datasets


In [2]:
# loading dataset
df = pd.read_csv("Telco.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
# renaming 'tenure' and 'gender'
df = df.rename(columns={'tenure': 'Tenure', 'gender': 'Gender'})

# converting 'TotalCharges' to numerical data type
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce', downcast='float')

# converting 'SeniorCitizen' to object data type
df['SeniorCitizen'] = df['SeniorCitizen'].astype(np.object)

# check
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   Gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   object 
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   Tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
# summarize duplicates
sum(df.duplicated('customerID'))
#df2 = df.drop_duplicates('customerID')

0

In [6]:
# remove customerID and PhoneService
df2 = df.drop(['customerID'], axis = 1)
df2.head(5)

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.850000,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.500000,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.150002,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.750000,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.649994,Yes


In [7]:
# Dummy Variables(One-Hot Encoding)
Gender = pd.get_dummies(df2['Gender'], prefix='Genger', drop_first=True)
Partner = pd.get_dummies(df2['Partner'], prefix='Partner', drop_first=True)
Dependents = pd.get_dummies(df2['Dependents'], prefix='Dependents', drop_first=True)
MultipleLines = pd.get_dummies(df2['MultipleLines'], prefix='MultipleLines', drop_first=True)
InternetService = pd.get_dummies(df2['InternetService'], prefix='InternetService', drop_first=True)
OnlineSecurity = pd.get_dummies(df2['OnlineSecurity'], prefix='OnlineSecurity', drop_first=True)
OnlineBackup = pd.get_dummies(df2['OnlineBackup'], prefix='OnlineBackup', drop_first=True)
DeviceProtection = pd.get_dummies(df2['DeviceProtection'], prefix='DeviceProtection', drop_first=True)
TechSupport = pd.get_dummies(df2['TechSupport'], prefix='TechSupport', drop_first=True)
StreamingTV = pd.get_dummies(df2['StreamingTV'], prefix='StreamingTV', drop_first=True)
StreamingMovies = pd.get_dummies(df2['StreamingMovies'], prefix='StreamingMovies', drop_first=True)
PaperlessBilling = pd.get_dummies(df2['PaperlessBilling'], prefix='PaperlessBilling', drop_first=True)
PaymentMethod = pd.get_dummies(df2['PaymentMethod'], prefix='PaymentMethod', drop_first=True)
Churn = pd.get_dummies(df2['Churn'], prefix='Churn', drop_first=True)
PaymentMethod = pd.get_dummies(df2['PhoneService'], prefix='PhoneService', drop_first=True)


df3 = pd.concat([df2, Gender, Partner, Dependents, MultipleLines, InternetService, 
                 OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, 
                 StreamingMovies, PaperlessBilling, PaymentMethod, Churn], axis=1)

In [8]:
# Label Encoding
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df3['Contract']= label_encoder.fit_transform(df3['Contract']) 

In [9]:
# drop original columns
list = ['Gender', 'Partner', 'Dependents', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 
'PaymentMethod', 'Churn', 'Contract', 'PhoneService']
df3.drop(df3[list], axis=1, inplace=True)
df3.head()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges,Genger_Male,Partner_Yes,Dependents_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,...,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,PaperlessBilling_Yes,PhoneService_Yes,Churn_Yes
0,0,1,29.85,29.850000,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,34,56.95,1889.500000,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,2,53.85,108.150002,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,0,45,42.30,1840.750000,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,2,70.70,151.649994,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [10]:
X=df3.drop('Churn_Yes',axis=1)
y= df3['Churn_Yes']

In [11]:
df3.head()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges,Genger_Male,Partner_Yes,Dependents_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,...,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,PaperlessBilling_Yes,PhoneService_Yes,Churn_Yes
0,0,1,29.85,29.850000,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,34,56.95,1889.500000,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,2,53.85,108.150002,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
3,0,45,42.30,1840.750000,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,2,70.70,151.649994,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [12]:
df3.columns

Index(['SeniorCitizen', 'Tenure', 'MonthlyCharges', 'TotalCharges',
       'Genger_Male', 'Partner_Yes', 'Dependents_Yes',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'PaperlessBilling_Yes', 'PhoneService_Yes', 'Churn_Yes'],
      dtype='object')

In [13]:
from sklearn.preprocessing import StandardScaler

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_df = pd.DataFrame(X,columns=X.columns)

In [14]:
# train test split
from sklearn.model_selection import train_test_split # split dataset
X_train, X_test, y_train, y_test = train_test_split(X_df,y,test_size=0.3,random_state=101)

In [15]:
#check
for i in [X_train, X_test, y_train, y_test]:
    i.index = range(i.shape[0]) 
    print(i.index)

RangeIndex(start=0, stop=4930, step=1)
RangeIndex(start=0, stop=2113, step=1)
RangeIndex(start=0, stop=4930, step=1)
RangeIndex(start=0, stop=2113, step=1)


In [16]:
#summarize missing values - X_train
X_train.isnull().sum()

SeniorCitizen                           0
Tenure                                  0
MonthlyCharges                          0
TotalCharges                            9
Genger_Male                             0
Partner_Yes                             0
Dependents_Yes                          0
MultipleLines_No phone service          0
MultipleLines_Yes                       0
InternetService_Fiber optic             0
InternetService_No                      0
OnlineSecurity_No internet service      0
OnlineSecurity_Yes                      0
OnlineBackup_No internet service        0
OnlineBackup_Yes                        0
DeviceProtection_No internet service    0
DeviceProtection_Yes                    0
TechSupport_No internet service         0
TechSupport_Yes                         0
StreamingTV_No internet service         0
StreamingTV_Yes                         0
StreamingMovies_No internet service     0
StreamingMovies_Yes                     0
PaperlessBilling_Yes              

In [17]:
# fill missing value w/ mean 
X_train['TotalCharges'].fillna(value=X_train['TotalCharges'].mean(), inplace=True)
# check missing values
X_train.isnull().sum()

SeniorCitizen                           0
Tenure                                  0
MonthlyCharges                          0
TotalCharges                            0
Genger_Male                             0
Partner_Yes                             0
Dependents_Yes                          0
MultipleLines_No phone service          0
MultipleLines_Yes                       0
InternetService_Fiber optic             0
InternetService_No                      0
OnlineSecurity_No internet service      0
OnlineSecurity_Yes                      0
OnlineBackup_No internet service        0
OnlineBackup_Yes                        0
DeviceProtection_No internet service    0
DeviceProtection_Yes                    0
TechSupport_No internet service         0
TechSupport_Yes                         0
StreamingTV_No internet service         0
StreamingTV_Yes                         0
StreamingMovies_No internet service     0
StreamingMovies_Yes                     0
PaperlessBilling_Yes              

In [18]:
#summarize missing values - y_train
y_train.isnull().sum()

0

In [19]:
#summarize missing values - X_test
X_test.isnull().sum()

SeniorCitizen                           0
Tenure                                  0
MonthlyCharges                          0
TotalCharges                            2
Genger_Male                             0
Partner_Yes                             0
Dependents_Yes                          0
MultipleLines_No phone service          0
MultipleLines_Yes                       0
InternetService_Fiber optic             0
InternetService_No                      0
OnlineSecurity_No internet service      0
OnlineSecurity_Yes                      0
OnlineBackup_No internet service        0
OnlineBackup_Yes                        0
DeviceProtection_No internet service    0
DeviceProtection_Yes                    0
TechSupport_No internet service         0
TechSupport_Yes                         0
StreamingTV_No internet service         0
StreamingTV_Yes                         0
StreamingMovies_No internet service     0
StreamingMovies_Yes                     0
PaperlessBilling_Yes              

In [20]:
# fill missing value w/ mean 
X_test['TotalCharges'].fillna(value=X_test['TotalCharges'].mean(), inplace=True)
# check missing values
X_test.isnull().sum()

SeniorCitizen                           0
Tenure                                  0
MonthlyCharges                          0
TotalCharges                            0
Genger_Male                             0
Partner_Yes                             0
Dependents_Yes                          0
MultipleLines_No phone service          0
MultipleLines_Yes                       0
InternetService_Fiber optic             0
InternetService_No                      0
OnlineSecurity_No internet service      0
OnlineSecurity_Yes                      0
OnlineBackup_No internet service        0
OnlineBackup_Yes                        0
DeviceProtection_No internet service    0
DeviceProtection_Yes                    0
TechSupport_No internet service         0
TechSupport_Yes                         0
StreamingTV_No internet service         0
StreamingTV_Yes                         0
StreamingMovies_No internet service     0
StreamingMovies_Yes                     0
PaperlessBilling_Yes              

In [21]:
#summarize missing values - y_test
y_test.isnull().sum()

0

In [22]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
4925    0
4926    0
4927    1
4928    1
4929    1
Name: Churn_Yes, Length: 4930, dtype: uint8

In [23]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate

In [24]:
oversampler=SMOTE(random_state=0)
smote_train, smote_target = oversampler.fit_resample(X_train,y_train)

In [25]:
import sklearn
from sklearn.svm import SVC
svc = SVC(kernel='rbf', C=1)
svc.fit(smote_train, smote_target)

SVC(C=1)

In [26]:
from sklearn.metrics import accuracy_score
y_pred= svc.predict(X_test)
svm_test_accuracy= sklearn.metrics.accuracy_score(y_test, y_pred)
print("Accuracy Rate of SVC: {:.2f}".format(svm_test_accuracy))

Accuracy Rate of SVC: 0.68


In [28]:
from sklearn import metrics

lm_svm = SVC(kernel='linear') 
lm_svm.fit(X_train,y_train)
preds = lm_svm.predict(X_test)
svm_test_accuracy_2= sklearn.metrics.accuracy_score(y_test, preds)
print("Accuracy Rate of SVC: {:.2f}".format(svm_test_accuracy_2))

Accuracy Rate of SVC: 0.80


In [29]:
pred_train = lm_svm.predict(X_train)
svm_train_accuracy= sklearn.metrics.accuracy_score(y_train, pred_train)
print("Accuracy Rate of SVC: {:.2f}".format(svm_train_accuracy))

Accuracy Rate of SVC: 0.79


In [30]:
# Evaluating model performance
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,preds))  
print(classification_report(y_test,preds))

[[1447   99]
 [ 316  251]]
              precision    recall  f1-score   support

           0       0.82      0.94      0.87      1546
           1       0.72      0.44      0.55       567

    accuracy                           0.80      2113
   macro avg       0.77      0.69      0.71      2113
weighted avg       0.79      0.80      0.79      2113



In [31]:

from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=4,scoring="accuracy", cv=10)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


GridSearchCV(cv=10, estimator=SVC(), n_jobs=4,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=3)

In [32]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
SVC(C=1, gamma=0.001)


In [33]:

grid_predictions = grid.predict(X_test)

# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.80      0.93      0.86      1546
           1       0.67      0.37      0.48       567

    accuracy                           0.78      2113
   macro avg       0.73      0.65      0.67      2113
weighted avg       0.77      0.78      0.76      2113



In [34]:
print('-----')
print(f'Best parameters {grid.best_params_}')
print(f'Mean cross-validated accuracy score of the best_estimator: {grid.best_score_:.3f}')
print('-----')

svm_best=grid.best_score_*100
svm_best

-----
Best parameters {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
Mean cross-validated accuracy score of the best_estimator: 0.779
-----


77.93103448275863